In [ ]:
!pip install numpy hist matplotlib mplhep pandas plotly scipy tqdm

In [14]:
import importlib
import sys
from tqdm import tqdm
import  os
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
hep.style.use([hep.style.ATLAS])

file_path = dir_path+'data//proAnubis_240716_2142.raw' # insert your file

In [8]:
importlib.reload(rawFileReader) # Reload fReader
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # reload in the classs object
order = [(0,1), (1,2), (2,3), (3,4)] # Order what you want to align
max_process_event_chunk = 100 # End the loop early
processedEvents = 0 # Initialisation


with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        #print(event_chunk[0].tdcEvents[0].time)
        pbar.update(1)

Processing Events: 100%|██████████| 100/100 [00:02<00:00, 39.18Events/s]


In [10]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # reload in the classs object
order = [(0,1), (1,2), (2,3), (3,4)] # Order what you want to align
max_process_event_chunk = 100 # End the loop early
processedEvents = 0 # Initialisation

#Initialise variables to store the results
mets = [[] for tdc in range(5)]
empty_header = 0 # Counting the number of empty headers
emtpy_events_with_header = [[], []] # prepare two lists of same dimensions for line graph plotting
tdc_event_count = [[] for tdc in range(5)]# prepare for histogram plotting


with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval) # get the aligned events
        tdc_event_count_buffer = RTools.find_tdc_event_count(event_chunk) # finding the number of headers
        [tdc_event_count[i].append(tdc_event_count_buffer[i]) for i in range(5)] # write from buffer to memory
        for idx, (i, j) in enumerate(order):
            x, y, l, m = ATools.find_tdc_alignment_metric(i, j) # determining which RPCs to use for aignment metric
            alignMet = ATools.calcAvgAlign(event_chunk, 0, x, y, l, m, i, j, processedEvents) # determine the alignment metric
            empty_header += (alignMet == -1) # Calculating the number of empty headers
            emtpy_events_with_header[0].append(empty_header) # write to memory
            emtpy_events_with_header[1].append(processedEvents)
            mets[idx].append(alignMet) # write to memory
        pbar.update(1)
    
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 8))

for idx, item in enumerate(order):
    met = mets[idx]
    i, j = item
    binsx = [x for x in range(len(met))]
    ax.plot(binsx, met, label=f'TDC{i} and TDC{j}, offset 0')


ax.set_xlim(0, max_process_event_chunk)
ax.set_ylim(-1, 40)
ax.legend()
ax.set_title('Alignment graph')
ax.set_ylabel('Average $\sqrt{d\eta^2+d\phi^2}$')
ax.set_xlabel('Processed Event chunks')
plt.show()

Processing Events:   0%|          | 0/100 [00:00<?, ?Events/s]

Processing Events: 100%|██████████| 100/100 [00:02<00:00, 34.45Events/s]


In [12]:
fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(5):
    met = tdc_event_count[tdc]
    binsx = [x for x in range(len(met))]
    ax.plot(binsx, met, label=f'TDC{tdc}')

ax.set_xlim(0, max_process_event_chunk)
# ax.set_ylim(-1, 100)
ax.legend()
ax.set_title('TDC number of events recorded')
ax.set_ylabel('num of events')
ax.set_xlabel('Processed Event')
plt.show()
    

In [13]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 150 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        if event_chunk:
            TAnalyser.update_event(event_chunk, processedEvents)
            status, failure = TAnalyser.check_eta_trigger()
        pbar.update(1)

Processing Events: 100%|██████████| 150/150 [00:03<00:00, 38.97Events/s]
